In [1]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable('json')

from sublimpy import turbulence
import matplotlib.pyplot as plt
from sublimpy import tidy
import metpy.constants

# Open data

## Open SOS Measurement Dataset

In [2]:
start_date = '20221130'
end_date = '20230509'
# open files
tidy_df_5Min_fn = f'tidy_df_{start_date}_{end_date}_noplanar_fit_clean.parquet'
tidy_df_5Min_fn_dirty = f'tidy_df_{start_date}_{end_date}_noplanar_fit.parquet'

tidy_df_5Min = pd.read_parquet(tidy_df_5Min_fn)
tidy_df_5Min_dirty = pd.read_parquet(tidy_df_5Min_fn_dirty)

# convert time column to datetime
tidy_df_5Min['time'] = pd.to_datetime(tidy_df_5Min['time'])
tidy_df_5Min_dirty['time'] = pd.to_datetime(tidy_df_5Min_dirty['time'])
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df_5Min = tidy_df_5Min.set_index('time').sort_index().loc[start_date:end_date].reset_index()
tidy_df_5Min_dirty = tidy_df_5Min_dirty.set_index('time').sort_index().loc[start_date:end_date].reset_index()

# Open z0 datasets - created with `calculate_roughness_length.ipynb`

In [3]:
z0_andreas_df = pd.read_parquet("z0estimates/z0_andreas_df.parquet")
z0_andreas_df_weekly = pd.read_parquet("z0estimates/z0_andreas_df_weekly.parquet")

In [4]:
time_series = tidy_df_5Min.query("variable == 'u*_3m_c'").time

z0_andreas_df = z0_andreas_df.set_index('time').reindex(time_series, method ='bfill')
z0_andreas_df_weekly = z0_andreas_df_weekly.set_index('time').reindex(time_series, method ='bfill')

In [5]:
tidy_df_5Min_dirty = tidy.tidy_df_add_variable(
    tidy_df_5Min_dirty,
    z0_andreas_df.z0,
    variable='z0_andreas',
    measurement='z0',
    height=3,
    tower='c'
)

tidy_df_5Min = tidy.tidy_df_add_variable(
    tidy_df_5Min,
    z0_andreas_df_weekly['all data'],
    variable='z0_andreas_weekly',
    measurement='z0',
    height=3,
    tower='c'
)


tidy_df_5Min_dirty = tidy.tidy_df_add_variable(
    tidy_df_5Min_dirty,
    z0_andreas_df.z0,
    variable='z0_andreas',
    measurement='z0',
    height=3,
    tower='c'
)

tidy_df_5Min_dirty = tidy.tidy_df_add_variable(
    tidy_df_5Min_dirty,
    z0_andreas_df_weekly['all data'],
    variable='z0_andreas_weekly',
    measurement='z0',
    height=3,
    tower='c'
)

In [6]:
src = tidy_df_5Min.query("measurement == 'z0'")
alt.Chart(src).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q").scale(type='log', domain=[1e-8, 1e-1], clamp=True),
    alt.Color('variable:O')
).properties(width=1000)

alt.Chart(...)

In [7]:
tidy_df_5Min.to_parquet(tidy_df_5Min_fn, index=False)
tidy_df_5Min_dirty.to_parquet(tidy_df_5Min_fn_dirty, index=False)

In [8]:
ls -lah | grep tidy | grep parquet

-rw-r--r--    1 elischwat  staff   257M Feb 20 22:02 tidy_df_20221130_20230509_noplanar_fit.parquet
-rw-r--r--    1 elischwat  staff   263M Feb 20 22:02 tidy_df_20221130_20230509_noplanar_fit_clean.parquet
-rw-r--r--    1 elischwat  staff    46M Feb 20 21:47 tidy_df_30min_20221130_20230509_noplanar_fit_clean.parquet
